# Predicting the Results for the remaining fixtures of the postponed Premier League 19-20 season
Due to the COVID-19 pandemic, a lot of football fans, like myself, have been upset due to the cancellation of the Premier League (and other leagues of course). Since I had a lot of free time now, I decided to try and predict the remaining fixtures of the English Premier League if it would have continued. <br>
<br>
I am aware now it might continue from June onwards but you can consider these results as what could have been if the league had continued without halt.<br><br>
I am only considering the performances of each team in the League so far, and not each player's performance. Please view the Assumptions to get a better understanding of what I'm getting at.<br>

If you are not interested in looking at the code, but just the final result, you can either scroll down or refer this blog post - https://medium.com/@joshanbabu96/predicting-the-remaining-premier-league-19-20-fixtures-1f41feaaf20f <br>

## Assumptions

These are the following assumptions:

1. <strong>Injury issues</strong><br>
If a player would have gotten injured in an upcoming fixture or missed many previous games due to injury but would have made it back by 11th March, that would certainly affect how the team would play, and thereby the result.
<br>However, I did not consider any of these changes.
<br>
2. <strong>Manager changes</strong><br>
Any possible manager changes which could affect playing styles of games will not be taken into account.
<br>A team’s performance is based on their overall performance in the league till now. So, if there was a manager who joined in between and changed the results thereafter, that doesn’t change how the team has been viewed.
<br>For instance, Carlo Ancelotti joined Everton on 21 December 2019 after which they have had much better results (5 wins out of 11 games as compared to 5 wins out of 18 games). Although they are playing better now, I have considered their pre-Ancelotti performance as well to determine where they stand in the league.
3. Any possibility of changes due to <strong>outside competitions</strong> (Champions League, Europa League, FA Cup, etc.) are also not taken into account<br>
4. Any <strong>tactical changes</strong> any team might perform in a game will also not be taken into account.
5. The difficulty for each match for each team is taken from the <strong>Fixture Difficulty Ratings (FDRs)</strong> from the Fantasy Premier League page.<br>
6. <strong>Rounding off numbers</strong><br>
There is obviously a lot of Math involved and the final scoreline can end up being something like 1.333–2.866 or similar. I was initially going to do a simple rounding off but I realized that a team with 2.8 goals doesn’t mean they would score 3 goals in a game but 2.99 means that they probably could. So, I set the limit at 0.9. If it is greater than 0.9, it will be rounded off to its ceiling value, else it will be the floor value.
    <br>Example:<br>
    1.333–2.866 = 1–2
    <br>0.95–1.1 = 1–1
    <br>2.90–2.91 = 2–3


## Steps

<strong>Note: </strong>Before I get to the steps that I used, I would like to say that this is my first ever project working with data. I did it without any help so it did take some time, and it might seem a bit long and some lines might be redundant. It would be great if you could provide some tips or advice that I can use on my next project. I'm still new to this so bear with me.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as ptl
import seaborn as sns
import plotly as py
import cufflinks as cf
import math

%matplotlib inline

I collected the data regarding Results of all Premier League games from 2000 to 2020 from Kaggle. I might work on some more interesting information in the future.<br>
Link: https://www.kaggle.com/irkaal/english-premier-league-results

In [3]:
df = pd.read_csv('EPL.csv')

In [4]:
df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
0,19/08/00,Charlton,Man City,4,0,H,2,0,H,Rob Harris,...,14,4,6,6,13,12,1,2,0,0
1,19/08/00,Chelsea,West Ham,4,2,H,1,0,H,Graham Barber,...,10,5,7,7,19,14,1,2,0,0
2,19/08/00,Coventry,Middlesbrough,1,3,A,1,1,D,Barry Knight,...,3,9,8,4,15,21,5,3,1,0
3,19/08/00,Derby,Southampton,2,2,D,1,2,A,Andy D'Urso,...,4,6,5,8,11,13,1,1,0,0
4,19/08/00,Leeds,Everton,2,0,H,2,0,H,Dermot Gallagher,...,8,6,6,4,21,20,1,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7381,07/03/2020,Wolves,Brighton,0,0,D,0,0,D,A Marriner,...,1,1,2,0,4,7,1,3,0,0
7382,07/03/2020,Burnley,Tottenham,1,1,D,1,0,H,J Moss,...,8,2,3,5,16,11,5,4,0,0
7383,08/03/2020,Chelsea,Everton,4,0,H,2,0,H,K Friend,...,11,1,6,1,8,10,1,2,0,0
7384,08/03/2020,Man United,Man City,2,0,H,1,0,H,M Dean,...,6,2,2,11,11,9,2,4,0,0


In [5]:
df.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF',
       'HY', 'AY', 'HR', 'AR'],
      dtype='object')

In [6]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

Since I need only the fixtures for the 19-20 Season and a Season starts from August onwards,

In [7]:
fixt1920 = df[df['Date'] > pd.Timestamp(2019,7,1)]

### **Creating the Results Table**

Requires:
Team Name, Games Played(GP), Games Won(GW), Games Tied(GT), Games Lost(GL), For Goals(GF), Against Goals(GA), Goal Difference(GD), Points(P)

In [8]:
ColumnNames = ['GP', 'GW', 'GT', 'GL', 'GF', 'GA', 'GD', 'P']
Table1920 = pd.DataFrame(index = fixt1920['HomeTeam'].unique(), columns = ColumnNames)
Table1920[ColumnNames] = 0
Table1920

,GP,GW,GT,GL,GF,GA,GD,P
Liverpool,0,0,0,0,0,0,0,0
West Ham,0,0,0,0,0,0,0,0
Bournemouth,0,0,0,0,0,0,0,0
Burnley,0,0,0,0,0,0,0,0
Crystal Palace,0,0,0,0,0,0,0,0
Watford,0,0,0,0,0,0,0,0
Tottenham,0,0,0,0,0,0,0,0
Leicester,0,0,0,0,0,0,0,0
Newcastle,0,0,0,0,0,0,0,0
Man United,0,0,0,0,0,0,0,0


<strong> The following is for predicting the standings by the end of the 19-20 season</strong>
<br>I will predict each match's results along with the Goals scored for and against.
<br>This is done by comparing the last 5 Fixtures of same difficulty using the Fixture Difficulty Ratings (FDRs) of each team.
<br>I can get the difficulty stats from Fantasy Premier League.
<br>I will compare the Goals Scored from the last 5 FDRs and find the average GF and GA to determine the scoreline. It will be rounded off for easier comparison.

In [9]:
for team in Table1920.index:
    for match in fixt1920.index:
        if(fixt1920.loc[match]['HomeTeam'] == team):
            Table1920.loc[team]['GP'] +=1
            Table1920.loc[team]['GF'] = Table1920.loc[team]['GF'] + fixt1920.loc[match]['FTHG']
            Table1920.loc[team]['GA'] = Table1920.loc[team]['GA'] + fixt1920.loc[match]['FTAG']
            if(fixt1920.loc[match]['FTR'] == 'H'):
                Table1920.loc[team]['GW'] +=1
                Table1920.loc[team]['P'] = Table1920.loc[team]['P'] + 3
            elif(fixt1920.loc[match]['FTR'] == 'A'):
                Table1920.loc[team]['GL'] +=1
            else:
                Table1920.loc[team]['GT'] +=1
                Table1920.loc[team]['P'] = Table1920.loc[team]['P'] + 1
        elif(fixt1920.loc[match]['AwayTeam'] == team):
            Table1920.loc[team]['GP'] +=1
            Table1920.loc[team]['GF'] = Table1920.loc[team]['GF'] + fixt1920.loc[match]['FTAG']
            Table1920.loc[team]['GA'] = Table1920.loc[team]['GA'] + fixt1920.loc[match]['FTHG']
            if(fixt1920.loc[match]['FTR'] == 'A'):
                Table1920.loc[team]['GW'] +=1
                Table1920.loc[team]['P'] = Table1920.loc[team]['P'] + 3
            elif(fixt1920.loc[match]['FTR'] == 'H'):
                Table1920.loc[team]['GL'] +=1
            else:
                Table1920.loc[team]['GT'] +=1
                Table1920.loc[team]['P'] = Table1920.loc[team]['P'] + 1

Table1920['GD'] = Table1920['GF'] - Table1920['GA']
Table1920 = Table1920.sort_values(['P', 'GD'], ascending=False)

Here, you can see the current Table as on 10 March 2020 which has remained the same till 27 May 2020 (and till whenever the league restarts). For confirmation, you can view the table on the <a href ="https://www.premierleague.com/tables">Premier League website</a> or on my <a href = "https://medium.com/@joshanbabu96/predicting-the-remaining-premier-league-19-20-fixtures-1f41feaaf20f">blog</a> (which took a screenshot of the table at that moment).

In [10]:
Table1920

,GP,GW,GT,GL,GF,GA,GD,P
Liverpool,29,27,1,1,66,21,45,82
Man City,28,18,3,7,68,31,37,57
Leicester,29,16,5,8,58,28,30,53
Chelsea,29,14,6,9,51,39,12,48
Man United,29,12,9,8,44,30,14,45
Wolves,29,10,13,6,41,34,7,43
Sheffield United,28,11,10,7,30,25,5,43
Tottenham,29,11,8,10,47,40,7,41
Arsenal,28,9,13,6,40,36,4,40
Burnley,29,11,6,12,34,40,-6,39


As you can see, each team has 9 games left except for Manchester City, Sheffield United, Arsenal and Aston Villa who have an extra game left each.<br>My objective is to predict how the table will end up by predicting the scorelines for the remaining games.

### Getting the difficulty of each fixture

To predict the score of each game, I need to check how each team performed against teams of similar difficulty. I have used current Fantasy Premier League's Fixture Difficulty Ratings (FDR) for this.
Now, I know FDR is based on the last 6 games and varies throughout the season, but I couldn't find the calculations for it, and thus, decided to use the current FDR ratings as seen in <a href="https://fantasy.premierleague.com/">Fantasy Premier League</a>.

In [11]:
RemFixt = pd.read_excel('Fixtures.xlsx', sheet_name = 'Remaining Fixtures')

In [12]:
RemFixt['Away Team'] = RemFixt['Away Team'].str.strip()
RemFixt['Home Team'] = RemFixt['Home Team'].str.strip()

<strong>Column Names for RemFixt:</strong> <br>
HomeTeam, AwayTeam, Gameweek, HomeFDR, AwayFDR, GFH, GAH, TeamsH, GFA, GAA, TeamsA, PEGH, PEGA, GH, GA, RGH, RGA <br>
Total Goals For in same FDR games for Home and Away - GFH and GFA <br>
Total Goals Against in same FDR games for Home and Away - GAH and GAA <br>
Number of teams played in same FDR for Home and Away - TeamsH and TeamsA <br>
In the Previous Encounter, Goals scored by the current Home and Away teams - PEGH and PEGA <br>
Home Team's Goal Average = ((GFH + PEGH)/TeamsH + GAA/TeamsA)/2 = GH <br>
Away Team's Goal Average = ((GFA + PEGA)/TeamsA + GAH/TeamsH)/2) + PEGA = GA <br>
Rounded Goals = RGH and RGA <br>
FDRs are for each team. That is, if Aston Villa is playing at home against Sheffield United, HomeFDR is the difficulty rating of the match for Aston Villa

In [13]:
RemFixt

,Home Team,Away Team,Gameweek,HomeFDR,AwayFDR
0,Aston Villa,Sheffield United,GW29,3,2
1,Manchester City,Arsenal,GW29,3,4
2,Aston Villa,Chelsea,GW30,4,2
3,Bournemouth,Crystal Palace,GW30,3,2
4,Brighton and Hove Albion,Arsenal,GW30,3,2
...,...,...,...,...,...
87,Leicester City,Manchester United,GW38,4,3
88,Manchester City,Norwich City,GW38,2,4
89,Newcastle United,Liverpool,GW38,4,2
90,Southampton,Sheffield United,GW38,3,2


In [14]:
AwayFDR = {
    'Liverpool': 5,
    'Arsenal': 4, 
    'Chelsea': 4,
    'Everton': 4,
    'Man City': 4, 
    'Man United': 4, 
    'Tottenham': 4,
    'Watford': 3,
    'Burnley': 3,
    'Leicester': 3,
    'Wolves': 3,
    'Crystal Palace': 3,
    'Sheffield United': 3,
    'Aston Villa': 2,
    'Bournemouth': 2,
    'Brighton': 2,
    'Newcastle': 2,
    'Norwich': 2,
    'West Ham': 2,
    'Southampton': 2
}

HomeFDR = {
    'Liverpool': 4,
    'Arsenal': 3, 
    'Chelsea': 4,
    'Everton': 2,
    'Man City': 4, 
    'Man United': 4, 
    'Tottenham': 3,
    'Watford': 2,
    'Burnley': 3,
    'Leicester': 3,
    'Wolves': 4,
    'Crystal Palace': 3,
    'Sheffield United': 3,
    'Aston Villa': 2,
    'Bournemouth': 2,
    'Brighton': 2,
    'Newcastle': 2,
    'Norwich': 2,
    'West Ham': 2,
    'Southampton': 3
}

Setting the previous games based on FDR for each team:

In [15]:
fixt1920['AwayFDR'] = fixt1920['HomeTeam'].map(AwayFDR)
fixt1920['HomeFDR'] = fixt1920['AwayTeam'].map(HomeFDR)

In [16]:
fixt1920

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HC,AC,HF,AF,HY,AY,HR,AR,AwayFDR,HomeFDR
7098,2019-08-09,Liverpool,Norwich,4,1,H,4,0,H,M Oliver,...,11,2,9,9,0,2,0,0,5,2
7099,2019-08-10,West Ham,Man City,0,5,A,0,1,A,M Dean,...,1,1,6,13,2,2,0,0,2,4
7100,2019-08-10,Bournemouth,Sheffield United,1,1,D,0,0,D,K Friend,...,3,4,10,19,2,1,0,0,2,3
7101,2019-08-10,Burnley,Southampton,3,0,H,0,0,D,G Scott,...,2,7,6,12,0,0,0,0,3,3
7102,2019-08-10,Crystal Palace,Everton,0,0,D,0,0,D,J Moss,...,6,2,16,14,2,1,0,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7381,2020-03-07,Wolves,Brighton,0,0,D,0,0,D,A Marriner,...,2,0,4,7,1,3,0,0,3,2
7382,2020-03-07,Burnley,Tottenham,1,1,D,1,0,H,J Moss,...,3,5,16,11,5,4,0,0,3,3
7383,2020-03-08,Chelsea,Everton,4,0,H,2,0,H,K Friend,...,6,1,8,10,1,2,0,0,4,2
7384,2020-03-08,Man United,Man City,2,0,H,1,0,H,M Dean,...,2,11,11,9,2,4,0,0,4,4


GFH, GAH, TeamsH, GFA, GAA, TeamsA, PEGH, PEGA, GH, GA, RGH, RGA

In [17]:
# CleaningNames of fixt1920 since names of RemFixt and fixt1920 are slightly different for some teams

Cleaned = {
    'Man City': 'Manchester City',
    'Brighton' : 'Brighton and Hove Albion',
    'Leicester' : 'Leicester City',
    'Man United' : 'Manchester United',
    'Newcastle' : 'Newcastle United',
    'Norwich' : 'Norwich City',
    'Tottenham' : 'Tottenham Hotspur',
    'West Ham' : 'West Ham United',
    'Wolves' : 'Wolverhampton Wanderers',
    'Arsenal': 'Arsenal'
}

fixt1920['HomeTeam'] = fixt1920['HomeTeam'].replace(Cleaned)
fixt1920['AwayTeam'] = fixt1920['AwayTeam'].replace(Cleaned)

In [18]:
# Functions for each column
def GFH(team, FDR):
    return fixt1920[(fixt1920['HomeTeam'] == team) & (fixt1920['HomeFDR'] == FDR)]['FTHG'].sum()

def GAH(team, FDR):
    return fixt1920[(fixt1920['HomeTeam'] == team) & (fixt1920['HomeFDR'] == FDR)]['FTAG'].sum()

def GFA(team, FDR):
    return fixt1920[(fixt1920['AwayTeam'] == team) & (fixt1920['AwayFDR'] == FDR)]['FTAG'].sum()

def GAA(team, FDR):
    return fixt1920[(fixt1920['AwayTeam'] == team) & (fixt1920['AwayFDR'] == FDR)]['FTHG'].sum()

def TeamsH(team, FDR):
    return (fixt1920[(fixt1920['HomeTeam'] == team) & (fixt1920['HomeFDR'] == FDR)]['HomeTeam'].count() + 1)

def TeamsA(team, FDR):
    return (fixt1920[(fixt1920['AwayTeam'] == team) & (fixt1920['AwayFDR'] == FDR)]['AwayTeam'].count() + 1)

def PEGH(HTeam, ATeam):
    return fixt1920[((fixt1920['AwayTeam'] == HTeam) & (fixt1920['HomeTeam'] == ATeam))]['FTAG'].sum()

def PEGA(HTeam, ATeam):
    return fixt1920[((fixt1920['AwayTeam'] == HTeam) & (fixt1920['HomeTeam'] == ATeam))]['FTHG'].sum()

In [19]:
# This is one place where I could use some help honestly.
# I couldn't fill the table since I think I had created a copy of the original which wasn't allowing me to change values.
# So, my solution was to create a list for each column, add the necessary values to the list, and then match them with their respected column.

GFHList = list()
GAHList = list()
GFAList = list()
GAAList = list()
TeamsHList = list()
TeamsAList = list()
PEGHList = list()
PEGAList = list()

for i in RemFixt.index:
    GFHList.append(GFH(RemFixt.loc[i]['Home Team'], RemFixt.loc[i]['HomeFDR']))
    GAHList.append(GAH(RemFixt.loc[i]['Home Team'], RemFixt.loc[i]['HomeFDR']))
    GFAList.append(GFA(RemFixt.loc[i]['Away Team'], RemFixt.loc[i]['AwayFDR']))
    GAAList.append(GAA(RemFixt.loc[i]['Away Team'], RemFixt.loc[i]['AwayFDR']))
    TeamsHList.append(TeamsH(RemFixt.loc[i]['Home Team'], RemFixt.loc[i]['HomeFDR']))
    TeamsAList.append(TeamsA(RemFixt.loc[i]['Away Team'], RemFixt.loc[i]['AwayFDR']))
    PEGHList.append(PEGH(RemFixt.loc[i]['Home Team'], RemFixt.loc[i]['Away Team']))
    PEGAList.append(PEGA(RemFixt.loc[i]['Home Team'], RemFixt.loc[i]['Away Team']))

RemFixt['GFH'] = GFHList
RemFixt['GAH'] = GAHList
RemFixt['GFA'] = GFAList
RemFixt['GAA'] = GAAList
RemFixt['TeamsH'] = TeamsHList
RemFixt['TeamsA'] = TeamsAList
RemFixt['PEGH'] = PEGHList
RemFixt['PEGA'] = PEGAList
RemFixt['GH'] = (((RemFixt['GFH'] + RemFixt['PEGH']) / (RemFixt['TeamsH'])) + ((RemFixt['GAA'] + RemFixt['PEGH']) / (RemFixt['TeamsA']))) / (2)
RemFixt['GA'] = (((RemFixt['GFA'] + RemFixt['PEGA']) / (RemFixt['TeamsA'])) + ((RemFixt['GAH'] + RemFixt['PEGA']) / (RemFixt['TeamsH']))) / (2)

<strong>Explaining the GH and GA formulae:</strong><br>
GH = ((Avg. of goals scored by Home Team) + (Avg. of goals conceded by Away Team)) / 2 <br>
GH = ( (Total no.of goals scored/No. of teams faced) + (Total no. of goals scored/No. of teams faced) ) / 2 <br>
GH = ( ((GFH + PEGH)/(TeamsH)) + ((GAA + PEGH)/(TeamsA)) ) / 2

As mentioned in the assumptions, I have rounded off based on decimal value of 0.9

In [20]:
def myRound(number):
    if (number - math.floor(number) <= 0.9):
        return math.floor(number)
    else:
        return round(number,0)

In [21]:
RemFixt['RGH'] = RemFixt['GH'].apply(myRound)
RemFixt['RGA'] = RemFixt['GA'].apply(myRound)

In [22]:
RemFixt

,Home Team,Away Team,Gameweek,HomeFDR,AwayFDR,GFH,GAH,GFA,GAA,TeamsH,TeamsA,PEGH,PEGA,GH,GA,RGH,RGA
0,Aston Villa,Sheffield United,GW29,3,2,6,12,5,3,5,5,0,2,0.900000,2.100000,0.0,2.0
1,Manchester City,Arsenal,GW29,3,4,11,6,3,3,6,4,3,0,1.916667,0.875000,2.0,0.0
2,Aston Villa,Chelsea,GW30,4,2,2,8,10,7,3,6,1,2,1.166667,2.666667,1.0,2.0
3,Bournemouth,Crystal Palace,GW30,3,2,2,3,5,4,4,6,0,1,0.583333,1.000000,0.0,1.0
4,Brighton and Hove Albion,Arsenal,GW30,3,2,4,7,7,4,7,5,2,1,1.028571,1.371429,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Leicester City,Manchester United,GW38,4,3,2,7,6,6,5,5,0,1,0.800000,1.500000,0.0,1.0
88,Manchester City,Norwich City,GW38,2,4,19,1,3,6,6,4,2,3,2.750000,1.083333,2.0,1.0
89,Newcastle United,Liverpool,GW38,4,2,5,3,10,2,5,6,1,3,0.850000,1.683333,0.0,1.0
90,Southampton,Sheffield United,GW38,3,2,3,12,5,3,5,5,1,0,0.800000,1.700000,0.0,1.0


### Creating the table for the Remaining Fixtures alone:

In [23]:
RemFixtTable = pd.DataFrame(index = Table1920.index, columns = ColumnNames)

#Cleaning the Team names of the table to match with the rest of the data
RemFixtTable = RemFixtTable.reset_index()
RemFixtTable['index'] = RemFixtTable['index'].replace(Cleaned)
RemFixtTable = RemFixtTable.set_index('index', drop=True)

RemFixtTable[ColumnNames] = 0

for team in RemFixtTable.index:
    for match in RemFixt.index:
        if(RemFixt.loc[match]['Home Team'] == team):
            RemFixtTable.loc[team]['GP'] +=1
            RemFixtTable.loc[team]['GF'] = RemFixtTable.loc[team]['GF'] + RemFixt.loc[match]['RGH']
            RemFixtTable.loc[team]['GA'] = RemFixtTable.loc[team]['GA'] + RemFixt.loc[match]['RGA']
            if(RemFixt.loc[match]['RGH'] > RemFixt.loc[match]['RGA']):
                RemFixtTable.loc[team]['GW'] +=1
                RemFixtTable.loc[team]['P'] = RemFixtTable.loc[team]['P'] + 3
            elif(RemFixt.loc[match]['RGH'] < RemFixt.loc[match]['RGH']):
                RemFixtTable.loc[team]['GL'] +=1
            else:
                RemFixtTable.loc[team]['GT'] +=1
                RemFixtTable.loc[team]['P'] = RemFixtTable.loc[team]['P'] + 1
        elif(RemFixt.loc[match]['Away Team'] == team):
            RemFixtTable.loc[team]['GP'] +=1
            RemFixtTable.loc[team]['GF'] = RemFixtTable.loc[team]['GF'] + RemFixt.loc[match]['RGA']
            RemFixtTable.loc[team]['GA'] = RemFixtTable.loc[team]['GA'] + RemFixt.loc[match]['RGH']
            if(RemFixt.loc[match]['RGA'] > RemFixt.loc[match]['RGH']):
                RemFixtTable.loc[team]['GW'] +=1
                RemFixtTable.loc[team]['P'] = RemFixtTable.loc[team]['P'] + 3
            elif(RemFixt.loc[match]['RGA'] < RemFixt.loc[match]['RGH']):
                RemFixtTable.loc[team]['GL'] +=1
            else:
                RemFixtTable.loc[team]['GT'] +=1
                RemFixtTable.loc[team]['P'] = RemFixtTable.loc[team]['P'] + 1
RemFixtTable['GD'] = RemFixtTable['GF'] - RemFixtTable['GA']

RemFixtTable = RemFixtTable.sort_values(['P', 'GD'], ascending = False)

RemFixtTable

,GP,GW,GT,GL,GF,GA,GD,P
index,,,,,,,,
Manchester City,10,8,2,0,18,7,11,26
Liverpool,9,8,1,0,14,3,11,25
Chelsea,9,6,2,1,14,9,5,20
Sheffield United,10,5,4,1,10,6,4,19
Wolverhampton Wanderers,9,5,3,1,10,5,5,18
Leicester City,9,5,3,1,8,5,3,18
Tottenham Hotspur,9,4,4,1,11,7,4,16
Manchester United,9,3,6,0,10,7,3,15
Arsenal,10,4,3,3,11,11,0,15


In [24]:
#Cleaning the Team names of the table to match with the rest of the data
Table1920 = Table1920.reset_index()
Table1920['index'] = Table1920['index'].replace(Cleaned)
Table1920 = Table1920.set_index('index', drop=True)

Table1920

,GP,GW,GT,GL,GF,GA,GD,P
index,,,,,,,,
Liverpool,29,27,1,1,66,21,45,82
Manchester City,28,18,3,7,68,31,37,57
Leicester City,29,16,5,8,58,28,30,53
Chelsea,29,14,6,9,51,39,12,48
Manchester United,29,12,9,8,44,30,14,45
Wolverhampton Wanderers,29,10,13,6,41,34,7,43
Sheffield United,28,11,10,7,30,25,5,43
Tottenham Hotspur,29,11,8,10,47,40,7,41
Arsenal,28,9,13,6,40,36,4,40


### For the Overall Table

In [25]:
Table1920Final = pd.concat([Table1920,RemFixtTable]).reset_index().groupby('index').sum().sort_values(['P','GD'], ascending = False)
Table1920Final

,GP,GW,GT,GL,GF,GA,GD,P
index,,,,,,,,
Liverpool,38,35,2,1,80.0,24.0,56.0,107
Manchester City,38,26,5,7,86.0,38.0,48.0,83
Leicester City,38,21,8,9,66.0,33.0,33.0,71
Chelsea,38,20,8,10,65.0,48.0,17.0,68
Sheffield United,38,16,14,8,40.0,31.0,9.0,62
Wolverhampton Wanderers,38,15,16,7,51.0,39.0,12.0,61
Manchester United,38,15,15,8,54.0,37.0,17.0,60
Tottenham Hotspur,38,15,12,11,58.0,47.0,11.0,57
Arsenal,38,13,16,9,51.0,47.0,4.0,55


### <strong> Interesting Points </strong>

1. Liverpool breaks multiple records including most points and wins with 107 points and 35 wins respectively.
The second-highest is Manchester City with 100 points in the 2017-18 season.

In [78]:
Table1920Final.iloc[0]

GP     38.0
GW     35.0
GT      2.0
GL      1.0
GF     80.0
GA     24.0
GD     56.0
P     107.0
Name: Liverpool, dtype: float64

2. Games where at least one team failed to score

In [42]:
OneTeamGoallessRemFixt = RemFixt[(RemFixt['RGH'] == 0) | (RemFixt['RGA'] == 0)]['Home Team'].count()
OneTeamGoallessfixt1920 = fixt1920[(fixt1920['FTHG'] == 0) | (fixt1920['FTAG'] == 0)]['HomeTeam'].count()
print("In the entire season:", OneTeamGoallessfixt1920 + OneTeamGoallessRemFixt)
print("Till season pause:", OneTeamGoallessfixt1920)
print("In remaining fixtures:", OneTeamGoallessRemFixt)

In the entire season: 171
Till season pause: 134
In remaining fixtures: 37


3. Number of Draws

In [50]:
DrawnGamesRemFixt = RemFixt[RemFixt['RGH'] == RemFixt['RGA']]['Home Team'].count()
DrawnGamesfixt1920 = fixt1920[fixt1920['FTHG'] == fixt1920['FTAG']]['HomeTeam'].count()
print("In the entire season:", DrawnGamesfixt1920 + DrawnGamesRemFixt)
print("Till season pause:", DrawnGamesfixt1920)
print("In remaining fixtures:", DrawnGamesRemFixt)

In the entire season: 109
Till season pause: 72
In remaining fixtures: 37


4. Highest Scoring Game from the remaining fixtures

In [73]:
# High scoring games
RemFixt[((RemFixt['RGH'] + RemFixt['RGA']) == (RemFixt['RGH'] + RemFixt['RGA']).max())]

,Home Team,Away Team,Gameweek,HomeFDR,AwayFDR,GFH,GAH,GFA,GAA,TeamsH,TeamsA,PEGH,PEGA,GH,GA,RGH,RGA
54,Brighton and Hove Albion,Manchester City,GW35,4,2,3,3,18,7,3,6,0,4,1.083333,3.0,1.0,3.0


5. Teams that have not lost a game in their remaining fixtures

In [75]:
RemFixtTable[RemFixtTable['GL'] == 0]

,GP,GW,GT,GL,GF,GA,GD,P
index,,,,,,,,
Manchester City,10,8,2,0,18,7,11,26
Liverpool,9,8,1,0,14,3,11,25
Manchester United,9,3,6,0,10,7,3,15
Southampton,9,1,8,0,8,10,-2,11


6. Teams that have not won a game in their remaining fixtures

In [76]:
RemFixtTable[RemFixtTable['GW'] == 0]

,GP,GW,GT,GL,GF,GA,GD,P
index,,,,,,,,
Aston Villa,10,0,9,1,9,16,-7,9
Everton,9,0,7,2,8,11,-3,7
Newcastle United,9,0,7,2,6,10,-4,7
Bournemouth,9,0,7,2,5,11,-6,7
Norwich City,9,0,5,4,7,12,-5,5
